In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_2D(a, mu, eta, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    mu_t = np.random.normal(mu, eta) 
    while np.linalg.norm(x, 2) < a(rt):
        x += mu_t*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta = np.arctan2(x[1], x[0])   
    
    return ndt+rt, theta

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.01, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_2D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    
    mu = np.array([prms[3], prms[4]])
    
    eta = prms[2]
    eta2 = eta**2
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                        dt=0.02, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            x0 =  prms[0]*np.cos(theta)
            x1 =  prms[0]*np.sin(theta)
            fixed = 1/(np.sqrt(eta2 * (rt - ndt) + 1))
            exponent0 = -0.5*mu[0]**2/eta2 + 0.5*(x0 * eta2 + mu[0])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent1 = -0.5*mu[1]**2/eta2 + 0.5*(x1 * eta2 + mu[1])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            term1 = fixed * np.exp(exponent0)
            term2 = fixed * np.exp(exponent1)
            
            density = term1 * term2 * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'eta_true': [],
               'eta_estimate':[],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': []}

min_threshold = 0.5
max_threshold = 3

min_ndt = 0.1
max_ndt = 1

min_eta = 0.1
max_eta = 1

min_mu = -3
max_mu = 3

In [6]:
for n in tqdm(range(5)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    eta = np.random.uniform(min_eta, max_eta)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['eta_true'].append(eta)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_2D(a, mu, eta, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_2D_likelihood,
                                     args=(RT, Theta),
                                     bounds=[(min_threshold, max_threshold), 
                                             (min_ndt, max_ndt), (min_eta, max_eta),
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['eta_estimate'].append(min_ans.x[2])
    recovery_df['mu1_estimate'].append(min_ans.x[3])
    recovery_df['mu2_estimate'].append(min_ans.x[4])
    
recovery_df = pd.DataFrame(recovery_df)

100%|████████████████████████████████████████████| 5/5 [35:25<00:00, 425.09s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,eta_true,eta_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
0,0.642283,0.688032,0.943701,0.946565,0.617838,0.996632,-1.371723,-1.710127,-1.093874,-1.183027
1,1.600373,1.639374,0.861402,0.867260,0.429113,0.440713,-0.449300,-0.487020,-0.507083,-0.557141
2,2.206849,2.251211,0.417237,0.449086,0.216167,0.313713,1.785663,1.818776,-2.317892,-2.556698
3,2.599708,3.000000,0.173172,0.100000,0.700426,0.906378,0.918378,1.043979,-1.121216,-1.419534
4,0.741459,0.803376,0.983615,0.983615,0.668329,1.000000,1.318673,1.674434,-2.095755,-2.237843


In [8]:
file_name = 'IE_2d_recovery_dvar.csv'
# old_recovery_data = pd.read_csv(file_name, index_col=0)
# recovery_df = pd.concat([old_recovery_data, 
#                          recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,eta_true,eta_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate
threshold_true,1.000000,0.992396,-0.942392,-0.930569,-0.317815,-0.493642,0.499550,0.462746,0.007793,-0.088900
threshold_estimate,0.992396,1.000000,-0.960133,-0.958173,-0.200083,-0.384667,0.480126,0.449192,0.038235,-0.064810
ndt_true,-0.942392,-0.960133,1.000000,0.996228,0.146521,0.252699,-0.511543,-0.468144,0.119755,0.227623
ndt_estimate,-0.930569,-0.958173,0.996228,1.000000,0.064173,0.184169,-0.482751,-0.445338,0.074444,0.185150
eta_true,-0.317815,-0.200083,0.146521,0.064173,1.000000,0.940306,-0.271234,-0.205585,0.286079,0.265793
eta_estimate,-0.493642,-0.384667,0.252699,0.184169,0.940306,1.000000,-0.269236,-0.219609,0.067271,0.063304
mu1_true,0.499550,0.480126,-0.511543,-0.482751,-0.271234,-0.269236,1.000000,0.995069,-0.770819,-0.811652
mu1_estimate,0.462746,0.449192,-0.468144,-0.445338,-0.205585,-0.219609,0.995069,1.000000,-0.746041,-0.784380
mu2_true,0.007793,0.038235,0.119755,0.074444,0.286079,0.067271,-0.770819,-0.746041,1.000000,0.993467
mu2_estimate,-0.088900,-0.064810,0.227623,0.185150,0.265793,0.063304,-0.811652,-0.784380,0.993467,1.000000
